# Interactive Dashboard -- State-wise visualization

In this analysis section, we virtualized the state-wise data by an interactive dashboard.

In [ ]:
import pandas as pd

def load_and_preprocess_data(file_path):
    """
    Load and preprocess the gun violence dataset.

    Parameters:
        file_path (str): Path to the CSV file containing the gun violence data.

    Returns:
        pd.DataFrame: Preprocessed DataFrame with 'Incident Date' as datetime and 'Year' extracted.
    """
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Convert 'Incident Date' to datetime
    data['Incident Date'] = pd.to_datetime(data['Incident Date'], errors='coerce')
    
    # Extract the year from the 'Incident Date'
    data['Year'] = data['Incident Date'].dt.year
    
    return data

Define the layout of the Dashboard. It structures the dashboard's interface: a dropdown for state selection and placeholders for statistic about each state and bar plots visualizing incidents by year, day of the week, and month.

In [ ]:
from dash import dcc, html

def create_app_layout(states):
    """
    Create the layout for the Dash application.

    Parameters:
        states (list): List of state names for the dropdown menu.

    Returns:
        html.Div: Dash application layout.
    """
    return html.Div([
        html.Div([
            # Dropdown for selecting a state
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in states],
                value=states[0],  # Default to the first state in the list
            ),
            html.Div(id='selected-state-output')
        ]),
        html.Div(id='state-statistics', style={'marginTop': '20px'}),
        html.Div([
            dcc.Graph(id='bar-plot-by-year'),
            dcc.Graph(id='bar-plot-by-week'),
            dcc.Graph(id='bar-plot-by-month')
        ])
    ])



Dynamically when a user selects a state from the dropdown and use *Plotly* to generate figures for yearly, weekly, and monthly incidents based on the selected state.

In [ ]:
import plotly.express as px
from dash.dependencies import Input, Output
from dash import html

def register_callbacks(app, data):
    """
    Register callbacks for the Dash app to update plots and statistics based on selected state.

    Parameters:
        app (dash.Dash): Dash application instance.
        data (pd.DataFrame): Gun violence dataset.
    """
    @app.callback(
        [Output("bar-plot-by-year", 'figure'),
         Output('bar-plot-by-week', 'figure'),
         Output('bar-plot-by-month', 'figure'),
         Output('state-statistics', 'children')],
        [Input('state-dropdown', 'value')]
    )
    def update_plots_and_stats(selected_state):
        """
        Update bar plots and statistics based on the selected state.

        Parameters:
            selected_state (str): State selected from the dropdown menu.

        Returns:
            tuple: Updated figures for year, week, month plots, and state-level statistics.
        """
        # Filter the data based on the selected state
        state_data = data[data['State'] == selected_state]

        # Yearly incidents
        yearly_incidents = state_data['Year'].value_counts().reset_index()
        yearly_incidents.columns = ['Year', 'Incidents']
        yearly_incidents = yearly_incidents.sort_values('Year')
        year_fig = px.bar(yearly_incidents, x='Year', y='Incidents', title=f'Gun Violence Incidents by Year in {selected_state}')

        # Weekly incidents
        incidents_by_day = state_data['Incident Date'].dt.day_name().value_counts().sort_index().reset_index()
        incidents_by_day.columns = ['Day of Week', 'Number of Incidents']
        week_fig = px.bar(incidents_by_day, x='Day of Week', y='Number of Incidents', title=f'Gun Violence Incidents by Day of Week in {selected_state}')

        # Monthly incidents
        incidents_by_month = state_data['Incident Date'].dt.month_name().value_counts().sort_index().reset_index()
        incidents_by_month.columns = ['Month', 'Number of Incidents']
        month_fig = px.bar(incidents_by_month, x='Month', y='Number of Incidents', title=f'Gun Violence Incidents by Month in {selected_state}')

        # State-level statistics
        total_accidents = len(state_data)
        total_deaths = state_data['Victims Killed'].sum()
        total_victims = state_data[['Victims Killed', 'Victims Injured']].sum().sum()

        state_stats = html.Div([
            html.P(f"Total Accidents: {total_accidents}"),
            html.P(f"Total Deaths: {total_deaths}"),
            html.P(f"Total Victims: {total_victims}")
        ])

        return year_fig, week_fig, month_fig, state_stats


This is the main driver script for running the interactive dashboard. It can:
- Initialize the Dash app.
- Load and preprocesses data using data_processing.py.
- Set the app layout using layout.py.
- Register callbacks using callback.py.
- Run the app server.

In [1]:
from dash import Dash
from file_loading import load_and_preprocess_data
from dash_layout import create_app_layout
from callbacks import register_callbacks

# Initialize the Dash app
app = Dash(__name__)

# Load and preprocess the data
file_path = 'ProcessedGunData.csv'  # Replace with the actual path
data = load_and_preprocess_data(file_path)

# Set up the app layout
app.layout = create_app_layout(data['State'].unique())

# Register callbacks
register_callbacks(app, data)

# Run the app
if __name__ == '__main__':
    """
    Run the Dash app server in debug mode.

    Usage:
        Execute this script to launch the Dash app. Ensure the dataset file is 
        accessible at the specified file path.
    """
    app.run_server(debug=True)